In [6]:
import numpy as np
from sklearn.preprocessing import minmax_scale
from scipy.spatial.distance import cdist
from pyts.metrics import dtw

## Features

In [7]:
def x_value(line):
    return line[1]


def y_value(line):
    return line[2]
 

def pressure_value(line):
    return line[3]


def vx_value(prev_line, current_line):
    x1 = prev_line[1]
    x2 = current_line[1]
    t1 = prev_line[0]
    t2 = current_line[0]

    if t2-t1 == 0:
        return 0
    return (x2-x1)/(t2-t1)


def vy_value(prev_line, current_line):
    y1 = prev_line[2]
    y2 = current_line[2]
    t1 = prev_line[0]
    t2 = current_line[0]
    if t2-t1 == 0:
        return 0
    return (y2-y1)/(t2-t1)


# Compute the "time serie", the features for each line of the signature file
def compute_features_vector(signature, normalize=True):
    rep = []
    first = True
    for line in signature:
        if first:
            prev_line = line
            first = False
        feature_vector = [x_value(line), y_value(line), pressure_value(
            line), vx_value(prev_line, line), vy_value(prev_line, line)]
        rep.append(feature_vector)
        prev_line = line

    array = np.asarray(rep)

    if normalize:
        array = minmax_scale(array, feature_range=(0, 1), axis=0)

    return array

def compute_dtw(signature1, signature2, normalize=True):

    feature_vector_1 = compute_features_vector(signature1, normalize)
    feature_vector_2 = compute_features_vector(signature2, normalize)

    dist_matrix = cdist(feature_vector_1, feature_vector_2)

    dtw_cost = dtw(precomputed_cost=dist_matrix, dist="precomputed",
                   method="sakoechiba")

    return dtw_cost


### Data preprocessing
We take the data contained in the enrollment txt files and store them in a dict (_users_signatures_), such that we reduce I/O calls in the rest of the code.

In [8]:
f = open('data/users.txt', 'r')
users = []
users_signatures = {}
counter = 0
for line in f:
    users.append(line[:-1])
print('Users: ',users)
for user in users:
    users_signatures[user] = []
    for i in range (1,6):
        f = open(f'data/enrollment/{user}-g-0{i}.txt', 'r')
        signature = []
        for line in f:
            values = np.asarray(line.split(),dtype=float)
            np.around(values,2)
            signature.append(values.tolist())
        users_signatures[user] += [signature]
        counter += 1

print(f'File treated: {counter} ({len(users)} users)')
#print(users_signatures['001'][1])

Users:  ['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030']
File treated: 150 (30 users)


### DTW values for enrollment
Here we compute the dtw value for each of the 5 signatures made by the users.

In [9]:
n_user = len(users)
dtw_values = {}
# Compute dtw values for the ground truth file (5 signatures/user)
for user in users_signatures:
    dtw_values[user] = []
    for i in range(1,6):
        for j in range(i+1, 5):
            f1 = users_signatures[user][i]
            f2 = users_signatures[user][j]
            dtw_values[user] += [compute_dtw(f1,f2)]

# Treshold taken from previous task
for key in dtw_values:
    values = dtw_values[key]
    mean = np.mean(values)
    maxVal = max(values)
    val = (9*mean + maxVal)/10
    dtw_values[key] = val

    
print(dtw_values)

{'001': 21.337495710141404, '002': 33.65679285287044, '003': 129.43584711667938, '004': 33.91739676695248, '005': 34.81167033848683, '006': 85.77734897310388, '007': 71.40198389624479, '008': 53.05671849758842, '009': 108.82358515411798, '010': 52.31629575127263, '011': 109.22350119191545, '012': 60.33896784970525, '013': 75.86280723026269, '014': 147.73017296505637, '015': 61.84098520902129, '016': 12.219038209910456, '017': 15.846105308522146, '018': 97.32186583446762, '019': 44.566866085921575, '020': 98.54756449119023, '021': 24.748092358531725, '022': 14.741133472404226, '023': 16.14948615549175, '024': 41.10004465992968, '025': 101.8370005471259, '026': 94.57086261029885, '027': 30.724742338473, '028': 53.99562333056313, '029': 16.836255355950463, '030': 91.28967752481599}


### Evaluate signatures